# Process CDs and get ALE Gene ref sequences of E COLI K12 STRAIN

In [12]:
from Bio import SeqIO

# Path to GenBank file
genbank_file = "bop27_1_4.gb"  # or .gbff

# Open and parse the GenBank file
with open(genbank_file, "r") as handle:
    for record in SeqIO.parse(handle, "genbank"):
        for feature in record.features:
            if feature.type == "CDS" and "translation" in feature.qualifiers:
                protein_id = feature.qualifiers.get("protein_id", ["no_id"])[0]
                product = feature.qualifiers.get("product", ["unnamed"])[0]
                translation = feature.qualifiers["translation"][0]
                #print(f">{protein_id} {product}")
                #print(translation)


# Read in Genes of interest from .gb file

In [13]:
from Bio import SeqIO

# Define the gene names you're interested in
target_genes = {"spoT", "topA", "yeiB"}

# Path to your GenBank file
genbank_file = "bop27_1_4.gb"  # replace with your filename

with open(genbank_file, "r") as handle:
    for record in SeqIO.parse(handle, "genbank"):
        for feature in record.features:
            if feature.type == "CDS":
                gene_name = feature.qualifiers.get("gene", [None])[0]
                if gene_name in target_genes:
                    protein_id = feature.qualifiers.get("protein_id", ["unknown_protein"])[0]
                    product = feature.qualifiers.get("product", ["unnamed"])[0]
                    translation = feature.qualifiers.get("translation", [""])[0]
                    print(f">{protein_id} | gene={gene_name} | product={product}")
                    print(translation)


>NP_415790.1 | gene=topA | product=DNA topoisomerase I, omega subunit
MGKALVIVESPAKAKTINKYLGSDYVVKSSVGHIRDLPTSGSAAKKSADSTSTKTAKKPKKDERGALVNRMGVDPWHNWEAHYEVLPGKEKVVSELKQLAEKADHIYLATDLDREGEAIAWHLREVIGGDDARYSRVVFNEITKNAIRQAFNKPGELNIDRVNAQQARRFMDRVVGYMVSPLLWKKIARGLSAGRVQSVAVRLVVEREREIKAFVPEEFWEVDASTTTPSGEALALQVTHQNDKPFRPVNKEQTQAAVSLLEKARYSVLEREDKPTTSKPGAPFITSTLQQAASTRLGFGVKKTMMMAQRLYEAGYITYMRTDSTNLSQDAVNMVRGYISDNFGKKYLPESPNQYASKENSQEAHEAIRPSDVNVMAESLKDMEADAQKLYQLIWRQFVACQMTPAKYDSTTLTVGAGDFRLKARGRILRFDGWTKVMPALRKGDEDRILPAVNKGDALTLVELTPAQHFTKPPARFSEASLVKELEKRGIGRPSTYASIISTIQDRGYVRVENRRFYAEKMGEIVTDRLEENFRELMNYDFTAQMENSLDQVANHEAEWKAVLDHFFSDFTQQLDKAEKDPEEGGMRPNQMVLTSIDCPTCGRKMGIRTASTGVFLGCSGYALPPKERCKTTINLVPENEVLNVLEGEDAETNALRAKRRCPKCGTAMDSYLIDPKRKLHVCGNNPTCDGYEIEEGEFRIKGYDGPIVECEKCGSEMHLKMGRFGKYMACTNEECKNTRKILRNGEVAPPKEDPVPLPELPCEKSDAYFVLRDGAAGVFLAANTFPKSRETRAPLVEELYRFRDRLPEKLRYLADAPQQDPEGNKTMVRFSRKTKQQYVSSEKDGKATGWSAFYVDGKWVEGKK
>NP_416657.1 | gene=yeiB | product=DUF418 family putative inner 

# Save to .faa file

In [14]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# Define the genes you want
target_genes = {"spoT", "topA", "yeiB"}

# GenBank file path
genbank_file = "bop27_1_4.gb"

# List to collect SeqRecords
records = []

# Parse the GenBank and extract target genes
with open(genbank_file, "r") as handle:
    for record in SeqIO.parse(handle, "genbank"):
        for feature in record.features:
            if feature.type == "CDS":
                gene_name = feature.qualifiers.get("gene", [None])[0]
                if gene_name in target_genes:
                    protein_id = feature.qualifiers.get("protein_id", ["unknown_protein"])[0]
                    product = feature.qualifiers.get("product", ["unnamed"])[0]
                    translation = feature.qualifiers.get("translation", [""])[0]
                    
                    # Create SeqRecord
                    seq_record = SeqRecord(
                        Seq(translation),
                        id=protein_id,
                        description=f"gene={gene_name} product={product}"
                    )
                    records.append(seq_record)

# Write to FASTA
output_file = "target_genes.faa"
SeqIO.write(records, output_file, "fasta")

print(f"Saved {len(records)} sequences to {output_file}")


Saved 3 sequences to target_genes.faa


# Find 5-kmer

In [41]:
from Bio import SeqIO

# Specify your FASTA file
fasta_file = "original_gene_fastas/yeiB.fasta"

# Read sequences from the file
sequences = list(SeqIO.parse(fasta_file, "fasta"))

# Check how many sequences you have
print(f"Loaded {len(sequences)} sequences.")

# Let's inspect the first sequence as an example
sequence = sequences[0].seq
print(f"\nSequence ID: {sequences[0].id}")
print(f"Sequence Length: {len(sequence)} amino acids\n")

# Index slicing example (positions are 1-based for biological clarity)
start_position = 143-2  # biological position (1-based)
end_position = 143+2    # inclusive

# Python uses 0-based indexing, so subtract 1 from start_position
kmer = sequence[start_position-1:end_position]

print(f"K-mer from position {start_position} to {end_position} (inclusive): {kmer}")


Loaded 1 sequences.

Sequence ID: NP_416657.1
Sequence Length: 385 amino acids

K-mer from position 141 to 145 (inclusive): VGLGV


# Mutate sequences that have 5-kmer and save all mutated seqs into new fasta file

In [42]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

# Your input FASTA file
input_fasta = "sequencesThatNeedMutated/yeiB_tomutSeq.fasta"

# Output FASTA file (mutated sequences)
output_fasta = "yeiB_mutated_seqs.fasta"

# The mutation in format 'OriginalAA_Position_MutatedAA', e.g., 'H33Y'
mutation = 'L143I'

# Parse the mutation string
original_aa = mutation[0]
mut_position = int(mutation[1:-1])  # numeric position
mutated_aa = mutation[-1]

# Define k-mer (centered around mutation site)
kmer = "VGLGV"

mutated_records = []

for record in SeqIO.parse(input_fasta, "fasta"):
    seq_str = str(record.seq)
    
    # Ensure sequence is long enough
    if len(seq_str) < mut_position:
        #print(f"❌ {record.id}: sequence too short for mutation position {mut_position}")
        continue

    # Check the amino acid matches original AA at given position
    if seq_str[mut_position - 1] != original_aa:
        #print(f"❌ {record.id}: original amino acid mismatch at position {mut_position} (found '{seq_str[mut_position - 1]}', expected '{original_aa}')")
        continue

    # Determine k-mer around mutation (if not explicitly defined)
    if not kmer:
        start = max(0, mut_position - flank - 1)
        end = min(len(seq_str), mut_position + flank)
        kmer_current = seq_str[start:end]
    else:
        kmer_current = kmer
    
    # Confirm k-mer exists in sequence
    kmer_pos = seq_str.find(kmer_current)
    if kmer_pos == -1:
        #print(f"❌ {record.id}: k-mer '{kmer_current}' not found in sequence.")
        continue
    
    # Apply mutation at the specific position
    mutated_seq = list(seq_str)
    mutated_seq[mut_position - 1] = mutated_aa
    mutated_seq = "".join(mutated_seq)

    # Make a new record for mutated sequence
    new_record = SeqRecord(
        Seq(mutated_seq),
        id=f"{record.id}_mut_{mutation}",
        description=f"Mutated {mutation} at position {mut_position}"
    )

    mutated_records.append(new_record)
    #print(f"✅ {record.id}: Mutation {mutation} applied successfully.")

# Save mutated sequences to FASTA file
SeqIO.write(mutated_records, output_fasta, "fasta")

print(f"\n🎯 Mutation completed. {len(mutated_records)} mutated sequences saved to '{output_fasta}'.")



🎯 Mutation completed. 2912 mutated sequences saved to 'yeiB_mutated_seqs.fasta'.


# Merge mutated seqs of each gene into one fasta file

In [43]:

# List of FASTA files you want to merge
fasta_files = ["topA_mutated_seqs.fasta", "yeiB_mutated_seqs.fasta", "spoT_mutated_seqs.fasta"]  
# Output merged FASTA file
merged_fasta = "ALE_dataset_sequences.fasta"

# Collect all sequences from the provided FASTA files
merged_records = []
for fasta in fasta_files:
    sequences = list(SeqIO.parse(fasta, "fasta"))
    print(f"📌 {fasta}: {len(sequences)} sequences loaded.")
    merged_records.extend(sequences)

# Write the merged sequences into a new FASTA file
SeqIO.write(merged_records, merged_fasta, "fasta")

print(f"\n✅ Successfully merged {len(merged_records)} sequences into '{merged_fasta}'.")


📌 topA_mutated_seqs.fasta: 4924 sequences loaded.
📌 yeiB_mutated_seqs.fasta: 2912 sequences loaded.
📌 spoT_mutated_seqs.fasta: 1526 sequences loaded.

✅ Successfully merged 9362 sequences into 'ALE_dataset_sequences.fasta'.


# Split dataset fasta files into training validation and test fastas

In [44]:
from Bio import SeqIO
from sklearn.model_selection import train_test_split

merged_fasta = "ALE_dataset_sequences.fasta"

# Load sequences from the merged file
all_sequences = list(SeqIO.parse(merged_fasta, "fasta"))
print(f"Total sequences loaded: {len(all_sequences)}")

# First, split into 80% train and 20% temporary set
train_seqs, temp_seqs = train_test_split(all_sequences, test_size=0.20, random_state=42)

# Now split the temporary set (20%) into 15% validation and 5% test
valid_seqs, test_seqs = train_test_split(temp_seqs, test_size=0.25, random_state=42)


# Save splits into separate FASTA files
SeqIO.write(train_seqs, "train.fasta", "fasta")
SeqIO.write(valid_seqs, "valid.fasta", "fasta")
SeqIO.write(test_seqs, "test.fasta", "fasta")

# Confirm your splits
print("\n✅ Data split completed:")
print(f"   🔹 Training set: {len(train_seqs)} sequences saved to 'train.fasta'")
print(f"   🔹 Validation set: {len(valid_seqs)} sequences saved to 'valid.fasta'")
print(f"   🔹 Test set: {len(test_seqs)} sequences saved to 'test.fasta'")


Total sequences loaded: 9362

✅ Data split completed:
   🔹 Training set: 7489 sequences saved to 'train.fasta'
   🔹 Validation set: 1404 sequences saved to 'valid.fasta'
   🔹 Test set: 469 sequences saved to 'test.fasta'
